In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/COVID-19_Radiography_Dataset"

In [ ]:
import numpy as np
import pandas as pd
import os

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

import seaborn as sns

from sklearn.metrics import classification_report, confusion_matrix
import glob
import matplotlib.pyplot as plt
import cv2

In [ ]:
train = "/content/drive/MyDrive/COVID-19_Radiography_Dataset/Train" 
val ="/content/drive/MyDrive/COVID-19_Radiography_Dataset/Val"
test = "/content/drive/MyDrive/COVID-19_Radiography_Dataset/Test"

gen_train = ImageDataGenerator(rescale = 1./255., zoom_range = 0.2,)
gen_val = ImageDataGenerator(rescale = 1./255.)
gen_test = ImageDataGenerator(rescale = 1./255.)

train_gen = gen_train.flow_from_directory(train, batch_size = 64, class_mode = 'categorical', target_size = (220, 220))
validation_gen = gen_val.flow_from_directory(val, batch_size = 64, class_mode = 'categorical', target_size=(220, 220))
test_gen = gen_test.flow_from_directory(test, shuffle = False, batch_size=64, class_mode = 'categorical', target_size=(220, 220))

Found 5786 images belonging to 2 classes.
Found 722 images belonging to 2 classes.
Found 722 images belonging to 2 classes.


In [ ]:
base_model = tf.keras.applications.ResNet50V2(weights = 'imagenet', input_shape = (220, 220,3), include_top = False)

for layer in base_model.layers:
    layer.trainable = False

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.15))
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense(2, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 7, 7, 2048)        23564800  
                                                                 
 global_average_pooling2d_1   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_3 (Dense)             (None, 512)               1049088   
                                                                 
 dropout_2 (Dropout)         (None, 512)               0         
                                                                 
 dense_4 (Dense)             (None, 256)               131328    
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                      

In [ ]:
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=["accuracy"])

callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=2,restore_best_weights=True)

history = model.fit(train_gen, validation_data = validation_gen, steps_per_epoch = 40, epochs = 25, callbacks = callback)

Epoch 1/25
40/40 [==============================] - 55s 1s/step - loss: 0.5217 - accuracy: 0.7914 - val_loss: 0.3325 - val_accuracy: 0.8227
Epoch 2/25
40/40 [==============================] - 47s 1s/step - loss: 0.3072 - accuracy: 0.8668 - val_loss: 0.3270 - val_accuracy: 0.8296
Epoch 3/25
40/40 [==============================] - 49s 1s/step - loss: 0.2581 - accuracy: 0.8875 - val_loss: 0.2565 - val_accuracy: 0.8670
Epoch 4/25
40/40 [==============================] - 47s 1s/step - loss: 0.2455 - accuracy: 0.9068 - val_loss: 0.1936 - val_accuracy: 0.9044
Epoch 5/25
40/40 [==============================] - 47s 1s/step - loss: 0.2389 - accuracy: 0.8961 - val_loss: 0.2261 - val_accuracy: 0.8837
Epoch 6/25
40/40 [==============================] - 48s 1s/step - loss: 0.2148 - accuracy: 0.9112 - val_loss: 0.1874 - val_accuracy: 0.9114
Epoch 7/25
40/40 [==============================] - 47s 1s/step - loss: 0.2153 - accuracy: 0.9051 - val_loss: 0.1576 - val_accuracy: 0.9321
Epoch 8/25
40/40 [==

In [ ]:
np.save("ResNet50V2-history1.npy",history.history)

In [ ]:
model.save("ResNet50V2_weights.h5")

In [ ]:
y_pred = model.predict(test_gen)

12/12 [==============================] - 303s 27s/step


In [ ]:
y_pred_list = []

for i in range(len(y_pred)):
    y_pred[i] = np.where(y_pred[i] == np.amax(y_pred[i]))
    y_pred_list.append(y_pred[i][0])

y_pred_list